In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

## Laden der Daten aus Datei: 

In [2]:
# open from file: 
with open("titel_pandemie_oa_pdf.xml", "r", encoding="utf-8") as f2:
    result = f2.readlines()

In [ ]:
#print(result[0:500])

Parsen des XML aus Datei:

In [4]:
content = "".join(result)
#print(content[0:500])

In [5]:
xml = soup(content, "lxml")

In [6]:
#print(xml.prettify()[:500])

Heraussuchen aller Datensätze des Typs "Bibliographic" aus dem XML:

In [7]:
myquery = xml.find_all('record', {'type':'Bibliographic'})
print(len(myquery))

773


In [ ]:
#print(myquery[1].prettify())


## Suche nach benötigten Informationen in den einzelnen Datensätzen: 

In [16]:
#Funktion für Titeldaten in MARC21

def parse_record_marc(item):

    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(item)))
    
    #idn
    idn = xml.findall("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'N/A' 
        
    
    #Titel 
    title = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    title2 = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'b']", namespaces=ns)
    
    if title and not title2:
        titletext = title[0].text
    elif title and title2:     
        titletext = title[0].text + " " + title2[0].text   #Hier wird extra nur mit einem Leerzeichen getrennt statt eines Separators
    else:
        titletext = "N/A"
    

    #date
    date = xml.findall("marc:controlfield[@tag = '008']", namespaces=ns)
    if date: 
        date = date[0].text
        date = date[7:11]
    else:
        date = 'N/A'

    
    #lang:
    lang = xml.findall("marc:datafield[@tag = '041']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        lang = lang[0].text
    except:
        lang = 'N/A'
    
    
    #URN: 
    for urn in xml.findall("marc:datafield[@tag = '024']/marc:subfield[@code = 'a']", namespaces=ns):
        if urn.text.startswith("urn:nbn"):
            urn = urn.text
        else: 
            urn = "N/A"


    #Links: 
    links = ""
    for link in xml.findall("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns):
        link = link.text + " "
        links += link
        
    
    
    meta_dict = {"idn":idn, "title": titletext, "date":date, "lang":lang, "urn":urn, "links":links} 
               
    return meta_dict
    

In [17]:
output = [parse_record_marc(record) for record in myquery]

In [18]:
df = pd.DataFrame(output)
df

,idn,title,date,lang,urn,links
0,131701071X,Auswirkungen der COVID-19-Pandemie auf die Pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,https://doi.org/10.11588/heidok.00034252 https...
1,1319384919,Auswirkungen der SARS-CoV-2-Pandemie auf die B...,2024,ger,urn:nbn:de:bvb:19-331095,https://doi.org/10.5282/edoc.33109 https://nbn...
2,1321172656,Erstes Working Paper zum Forschungsprojekt Lok...,2024,ger,urn:nbn:de:bvb:m347-opus-4868,https://doi.org/10.60948/OPUS-486 https://nbn-...
3,1318286735,Evaluation des psychischen Belastungserlebens ...,2024,ger,urn:nbn:de:bvb:20-opus-348164,https://doi.org/10.25972/OPUS-34816 https://nb...
4,1323400958,Förderung der organisationalen Resilienz touri...,2024,ger,urn:nbn:de:bvb:m347-opus-4876,https://nbn-resolving.org/urn:nbn:de:bvb:m347-...
...,...,...,...,...,...,...
768,1266845550,The impact of COVID-19 pandemic on nosocomia...,2022,eng,urn:nbn:de:0183-dgkh0004182,https://doi.org/10.3205/dgkh000418 https://nbn...
769,1227577281,TIP - Training of Intensive medical support in...,2021,eng,urn:nbn:de:0183-zma0014196,https://doi.org/10.3205/zma001419 https://nbn-...
770,1289022720,Understanding of professionalism among medical...,2023,eng,urn:nbn:de:0183-zma0016053,https://doi.org/10.3205/zma001605 https://nbn-...
771,123616332X,„Unsere Klienten waren die ersten Verlierer de...,2021,ger,urn:nbn:de:101:1-2021062812430983681376,https://nbn-resolving.org/urn:nbn:de:101:1-202...


In [19]:
df.to_hdf("pandemie_tit_oa_pdf.h5", key="df")